# Day 14: Restroom Redoubt ---

One of The Historians needs to use the bathroom; fortunately, you know there's a bathroom near an unvisited location on their list, and so you're all quickly teleported directly to the lobby of Easter Bunny Headquarters.

Unfortunately, EBHQ seems to have "improved" bathroom security again after your last visit. The area outside the bathroom is swarming with robots!

To get The Historian safely to the bathroom, you'll need a way to predict where the robots will be in the future. Fortunately, they all seem to be moving on the tile floor in predictable straight lines.

You make a list (your puzzle input) of all of the robots' current positions (p) and velocities (v), one robot per line. For example:

```
p=0,4 v=3,-3
p=6,3 v=-1,-3
p=10,3 v=-1,2
p=2,0 v=2,-1
p=0,0 v=1,3
p=3,0 v=-2,-2
p=7,6 v=-1,-3
p=3,0 v=-1,-2
p=9,3 v=2,3
p=7,3 v=-1,2
p=2,4 v=2,-3
p=9,5 v=-3,-3
```

Each robot's position is given as p=x,y where x represents the number of tiles the robot is from the left wall and y represents the number of tiles from the top wall (when viewed from above). So, a position of p=0,0 means the robot is all the way in the top-left corner.

Each robot's velocity is given as v=x,y where x and y are given in tiles per second. Positive x means the robot is moving to the right, and positive y means the robot is moving down. So, a velocity of v=1,-2 means that each second, the robot moves 1 tile to the right and 2 tiles up.

The robots outside the actual bathroom are in a space which is 101 tiles wide and 103 tiles tall (when viewed from above). However, in this example, the robots are in a space which is only 11 tiles wide and 7 tiles tall.

The robots are good at navigating over/under each other (due to a combination of springs, extendable legs, and quadcopters), so they can share the same tile and don't interact with each other. Visually, the number of robots on each tile in this example looks like this:

```
1.12.......
...........
...........
......11.11
1.1........
.........1.
.......1...
```

In [110]:
import numpy as np

class Robot:
    def __init__(self, pos, vector, layout):
        self.start = np.array(pos, dtype=int)
        self.position = self.start
        self.vector = np.array(vector, dtype=int)
        self.layout = np.array(layout, dtype=int)

    def reset(self):
        self.position = self.start

    def move(self, steps):
        self.position = (self.position + steps * self.vector) % self.layout

    @property
    def quadrant(self):
        middle = self.layout // 2
        if self.position[0] < middle[0] and self.position[1] < middle[1]:
            return 0

        if self.position[0] < middle[0] and self.position[1] > middle[1]:
            return 1

        if self.position[0] > middle[0] and self.position[1] < middle[1]:
            return 2

        if self.position[0] > middle[0] and self.position[1] > middle[1]:
            return 3

        return None

    def add_to_layout(self, layout, dot=False):
        if dot:
            c = '*'
        else:
            c = layout[self.position[0], self.position[1]]
            c = '1' if c == '.' else chr(ord(c) + 1)

        layout[self.position[0], self.position[1]] = c

    def __str__(self):
        return f"Robot at {self.start} moving {self.vector}"

    def __repr__(self):
        return self.__str__()


These robots have a unique feature for maximum bathroom security: they can teleport. When a robot would run into an edge of the space they're in, they instead teleport to the other side, effectively wrapping around the edges. Here is what robot `p=2,4 v=2,-3` does for the first few seconds:

```
Initial state:
...........
...........
...........
...........
..1........
...........
...........

After 1 second:
...........
....1......
...........
...........
...........
...........
...........

After 2 seconds:
...........
...........
...........
...........
...........
......1....
...........

After 3 seconds:
...........
...........
........1..
...........
...........
...........
...........

After 4 seconds:
...........
...........
...........
...........
...........
...........
..........1

After 5 seconds:
...........
...........
...........
.1.........
...........
...........
...........
```

In [111]:
def create_map(layout):
    return np.full(layout, '.')

def map_to_string(map, quadrants=None):
    if quadrants:
        map = map.copy()
        map[:, map.shape[1] // 2] = '|'
        map[map.shape[0] // 2, :] = '-'

    return '\n'.join([''.join(row) for row in map])

def example1(step_individual=True):
    # Row order major
    ex_layout = [7, 11]
    ex_robot = Robot([4,2], [-3, 2], ex_layout)


    if step_individual:
        for _ in range(5):
            ex_robot.move(1)
            map = create_map(ex_layout)

            ex_robot.add_to_layout(map)
            print(map_to_string(map))
            print()

    else:
        ex_robot.move(5)
        map = create_map(ex_layout)
        ex_robot.add_to_layout(map)
        print(map_to_string(map))


example1(True)
example1(False)

...........
....1......
...........
...........
...........
...........
...........

...........
...........
...........
...........
...........
......1....
...........

...........
...........
........1..
...........
...........
...........
...........

...........
...........
...........
...........
...........
...........
..........1

...........
...........
...........
.1.........
...........
...........
...........

...........
...........
...........
.1.........
...........
...........
...........


The Historian can't wait much longer, so you don't have to simulate the robots for very long. Where will the robots be after 100 seconds?

In the above example, the number of robots on each tile after 100 seconds has elapsed looks like this:

```
......2..1.
...........
1..........
.11........
.....1.....
...12......
.1....1....
```

To determine the safest area, count the number of robots in each quadrant after 100 seconds. Robots that are exactly in the middle (horizontally or vertically) don't count as being in any quadrant, so the only relevant robots are:

```
..... 2..1.
..... .....
1.... .....
           
..... .....
...12 .....
.1... 1....
```

In this example, the quadrants contain 1, 3, 4, and 1 robot. Multiplying these together gives a total safety factor of 12.

In [112]:


from functools import reduce
from typing import Counter


def load_robots(filename, layout):
    robots = []
    with open(filename, encoding='ascii') as f:
        for line in f:
            split = line.strip().split()

            pos = split[0].strip('p=').split(',')
            vector = split[1].strip('v=').split(',')

            # the file uses column major order, so swap the coordinates
            pos.reverse()
            vector.reverse()

            robots.append(Robot(pos, vector, layout))

    return robots

def part1(robots, steps):
    for robot in robots:
        robot.move(steps)

    counts = Counter(r.quadrant for r in robots)
    del counts[None]

    safety_factor = reduce(lambda x, y: x * y, counts.values())
    return safety_factor

EXAMPLE_LAYOUT = [7, 11]
EXAMPLE_INPUT = load_robots('14_example.txt', EXAMPLE_LAYOUT)
print("Example:", part1(EXAMPLE_INPUT, 100))



Example: 12


Predict the motion of the robots in your list within a space which is 101 tiles wide and 103 tiles tall. What will the safety factor be after exactly 100 seconds have elapsed?

In [113]:
LAYOUT = [103, 101]
INPUT = load_robots('14_input.txt', LAYOUT)
print(f"Part 1: {part1(INPUT, 100):,}")

Part 1: 231,221,760


In [104]:
from tqdm import tqdm


def print_at_step(robots, layout, i):
    print(i)
    map = create_map(layout)
    for robot in robots:
        robot.reset()
        robot.move(i)
        robot.add_to_layout(map, dot=True)

    print(map_to_string(map))
    print()

def part2(robots, layout):
    factors = []
    for i in tqdm(range(layout[0] * layout[1])):
        for robot in robots:
            robot.reset()

        factors.append(part1(robots, i))

    factors = np.array(factors)
    print(f"min: {factors.min():,} at {factors.argmin()}")
    print(f"max: {factors.max():,} at {factors.argmax()}")

    print_at_step(robots, layout, factors.argmin())
    print_at_step(robots, layout, factors.argmax())




part2(INPUT, LAYOUT)

100%|██████████| 100000/100000 [01:38<00:00, 1016.94it/s]
